In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%ls
%cd CNNtention/

 Volume in drive D is New Volume
 Volume Serial Number is 8AC3-D420

 Directory of D:\OMSCS\CS7643 Deep Learning\CNNtention\notebooks\Untuned\run4 - CBAM - With Residual Connections

09-12-2024  01:58    <DIR>          .
07-12-2024  09:06    <DIR>          ..
07-12-2024  15:41            34,317 cbam_resnet20_test_err.png
07-12-2024  15:41            47,471 cbam_resnet20_train_and_test_err.png
07-12-2024  15:41            34,095 cbam_resnet20_train_err.png
07-12-2024  15:41            32,534 cbam_resnet20_train_val_loss.png
05-12-2024  07:21    <DIR>          data
09-12-2024  01:58             5,492 run4.ipynb
               5 File(s)        153,909 bytes
               3 Dir(s)  174,726,193,152 bytes free
[WinError 2] The system cannot find the file specified: 'CNNtention/'
D:\OMSCS\CS7643 Deep Learning\CNNtention\notebooks\Untuned\run4 - CBAM - With Residual Connections


C:\Users\trath\miniconda3\envs\cs7643-final-project\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
import pipeline_template, mlflow, torch, torchvision
from experiment_configs import configs, ExperimentConfig, ModelConfig, DataConfig
from models.resnet_cbam import ResnetCBAM
from models.original_basic_block import OriginalBasicBlock
from skorch.callbacks import LRScheduler
from skorch.dataset import ValidSplit
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.transforms import transforms
mlflow.set_tracking_uri('file:./mlruns')

In [5]:
config={"cifar10_resnet20_cbam_training": lambda: ExperimentConfig(
        "CBAM ResNet20",
        DataConfig(name='CIFAR-10',
                   test_size=10000,
                   train_transform=transforms.Compose([
                transforms.RandomHorizontalFlip(0.5),
                transforms.RandomCrop(32, padding=4),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4918687901200927, 0.49185976472299225, 0.4918583862227116], std=[0.24697121702736, 0.24696766978537033, 0.2469719877121087])
            ]),
            test_transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4918687901200927, 0.49185976472299225, 0.4918583862227116], std=[0.24697121702736, 0.24696766978537033, 0.2469719877121087])
            ])),
        ModelConfig(model=ResnetCBAM(OriginalBasicBlock,[3, 3, 3], 10, 8),
                    lr=0.005,
                    optimizer=torch.optim.Adam,
                    batch_size=128,
                    max_epochs=64000//(45000 // 128),
                    weight_decay=0.0001,
                    momentum=0.9,
                    use_reg=True,
                    train_split=ValidSplit(cv=0.1),
                    scheduler=LRScheduler(
                        policy=MultiStepLR,
                        milestones=[
                            16000 // (45000 // 128), # == 45
                            32000 // (45000 // 128), # == 91
                            48000 // (45000 // 128) # == 136
                        ],
                        gamma=0.1 # this is the multiplication factor ("divide it by 10")
                    ),
                    add_test_set_eval=False,
                    use_early_stopping=False,
                    log_model=True)
    )}

In [6]:
pipeline_template.main('cifar10_resnet20_cbam_training',  True, config)

2024/12/09 01:59:57 INFO mlflow.tracking.fluent: Experiment with name 'cbam_resnet20' does not exist. Creating a new experiment.


=== Model Details ===
ResnetCBAM(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer1): Sequential(
    (0): OriginalBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
    )
    (1): OriginalBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1

  0%|          | 0/392 [00:00<?, ?it/s]

TypeError: '<' not supported between instances of 'NoneType' and 'float'